# 출처: https://www.philschmid.de/gptq-llama

# optimum과 GPTQ로 오픈 LLM 양자화

허깅 페이스 Optimum 팀은 AutoGPTQ 라이브러리로 언어 모델에 GPTQ 양자화를 적용하는 간단한 API를 제공하고 있습니다. GPTQ 양자화를 활용하면 오픈 LLM을 8비트, 4비트, 3비트, 심지어 2비트로 압축할 수 있어 성능 저하 없이 더 작은 하드웨어에서 실행할 수 있습니다.

이 블로그에서는 다음과 같은 내용을 배울 수 있습니다.

1. 개발 환경 설정
2. 양자화 데이터 세트 준비
3. 모델 로드 및 양자화
4. 성능 및 추론 속도 테스트
5. 보너스: 텍스트 생성 추론으로 추론 실행
  
시작하기 전에 GPTQ가 무엇인지 간단히 살펴보겠습니다.

_참고: 이 튜토리얼은 엔비디아 A10G GPU를 포함한 g5.2xlarge AWS EC2 인스턴스에서 작성하고 실행되었습니다._


## GPTQ란 무엇인가요?

[GPTQ](https://arxiv.org/abs/2210.17323)는 GPT와 같은 LLM을 압축하기 위한 후처리 양자화 방법입니다. GPTQ는 GPT 모델의 각 가중치를 32비트에서 3-4비트로 줄여서 모델을 압축합니다. 이로 인해 모델의 메모리 사용량이 크게 줄어들어 더 작은 하드웨어에서도 실행할 수 있게 됩니다. GPTQ는 모델의 각 레이어를 개별적으로 분석하고 가중치를 근사화하여 전체적인 정확도를 유지합니다.

주요 이점은 다음과 같습니다.
* 모델의 레이어를 16비트 대신 4비트로 양자화하여 필요한 메모리를 4배 줄입니다.
* 양자화는 점진적으로 수행되어 양자화로 인한 정확도 손실을 최소화합니다.
* fp16 모델과 동일한 지연 시간을 유지하면서 메모리 사용량은 4배 줄어들며, 경우에 따라 사용자 정의 커널로 인해 더 빠를 수도 있습니다.(예: [Exllama](https://github.com/turboderp/exllama))
* 양자화된 가중치는 미리 양자화된 상태로 디스크에 저장할 수 있습니다.

_참고: 현재 GPTQ 양자화는 텍스트 모델에만 적용됩니다. 또한 양자화 과정에는 많은 시간이 소요될 수 있습니다. 활용하려는 모델의 GPTQ 양자화 버전이 이미 있는지 [허깅 페이스 허브](https://huggingface.co/models?search=gptq)에서 확인할 수 있습니다._

--- 

## 1. 개발 환경 설정

이제 코딩을 시작하겠습니다. 우선, 종속성을 설치해 보겠습니다.

In [ ]:
!pip install "torch==2.0.1" "transformers==4.32.1" "optimum==1.12.0" "auto-gptq==0.4.2" "accelerate==0.22.0" "safetensors>=0.3.1" --upgrade

## 2. 양자화 데이터 세트 준비

GPTQ는 후처리 양자화 방법이므로 모델을 양자화하기 위해 데이터 세트를 준비해야 합니다. [허깅 페이스 허브](https://huggingface.co/datasets)의 데이터 세트나 자신의 데이터 세트를 활용할 수 있습니다. 이 블로그에서는 허깅 페이스 허브의 [WikiText](https://huggingface.co/datasets/wikitext) 데이터 세트를 활용할 것입니다. 이 데이터 세트는 가중치를 양자화하여 성능 손실을 최소화하는 데 활용됩니다. 양자화 데이터 세트는 최소한 128개의 샘플을 포함하는 것이 권장됩니다.

_참고: [TheBloke](https://huggingface.co/TheBloke)라는 매우 활동적인 커뮤니티 멤버가 허깅 페이스 허브에 수백 개의 GPTQ 가중치를 기여하고 있습니다. 그는 주로 일반 도메인 모델의 양자화 데이터 세트로 wikitext를 활용합니다._

예를 들어, 자신의 파인튜닝 데이터 세트를 양자화에 활용하고 싶다면 "id" 대신 리스트로 제공할 수 있습니다. 자세한 사항은 이 [예시](https://colab.research.google.com/drive/1_TIrmuKOFhuRRiTWN94iLKUFu6ZX4ceb)에서 확인할 수 있습니다.

In [ ]:
# 허깅 페이스에 있는 데이터 세트 id

dataset_id = "wikitext2"

## 3. 모델 로드 및 양자화

Optimum은 `optimum.qptq` 네임스페이스에서 GPTQ 양자화를 통합하고 있으며, `GPTQQuantizer`를 제공합니다. 이 양자화기는 데이터 세트(아이디 또는 리스트), 비트 수, 및 모델 시퀀스 길이(model_seqlen)를 입력으로 받습니다. 추가적인 커스터마이징이 필요한 경우, 자세한 사항은 [여기](https://github.com/huggingface/optimum/blob/234a427450a7dcc978b227fa627ebcdab1764318/optimum/gptq/quantizer.py#L76)에서 확인할 수 있습니다.

In [ ]:
from optimum.gptq import GPTQQuantizer

# GPTQ 양자화기
quantizer = GPTQQuantizer(bits=4, dataset=dataset_id, model_seqlen=4096)
quantizer.quant_method = "gptq"

양자화기를 생성한 후, 트랜스포머를 활용하여 모델을 로드할 수 있습니다. 예를 들어, 우리는 다른 블로그 포스트 ["확장 가이드: Instruction-tune Llama 2"](https://www.philschmid.de/instruction-tune-llama-2)에서 학습한 [Llama 2 7B](https://huggingface.co/philschmid/llama-2-7b-instruction-generator) 모델을 양자화할 것입니다. 우리는 모델을 fp16 형식으로 로드할 예정인데, GPTQ는 mixed int4/fp16 양자화 방식을 채택하여 가중치는 int4로 양자화되고 활성화는 float16으로 유지됩니다.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 허깅 페이스 모델 id
model_checkpoint = "NousResearch/Llama-2-7b-hf" # 비게이트(non-gated)
#model_checkpoint = "meta/llama-2-7b" # 게이트(gated)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False) # 빠른 토크나이저의 버그
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, low_cpu_mem_usage=True, torch_dtype=torch.float16) # 우리는 모델을 의도적으로 fp16으로 로드합니다.

모델을 로드하면 양자화할 준비가 완료됩니다.
_참고: 양자화 과정은 하드웨어에 따라 많은 시간이 소요될 수 있습니다. 이 예시에서는 A10G GPU 한 대에서 7B 모델의 양자화가 약 ~분이 걸렸습니다._ 

In [ ]:
import os 
import json

# 모델을 양자화합니다.
quantized_model = quantizer.quantize_model(model, tokenizer)

# 양자화된 모델을 디스크에 저장합니다.
save_folder = "quantized_llama"
model.save_pretrained(save_folder, safe_serialization=True)

# 새롭고 빠른 토크나이저를 로드하고 디스크에 저장합니다.
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint).save_pretrained(save_folder)

# TGI를 위한 quantize_config.json을 저장합니다.
with open(os.path.join(save_folder, "quantize_config.json"), "w", encoding="utf-8") as f:
  quantizer.disable_exllama = False
  json.dump(quantizer.to_dict(), f, indent=2)

모델이 부분적으로 오프로드되었으므로 오류를 피하기 위해 `disable_exllama`를 `True`로 설정합니다. 추론 및 운영 환경에 로드할 때는 Exllama 커널을 활용하기 위해 `config.json`을 변경해야 합니다.

In [ ]:
with open(os.path.join(save_folder, "config.json"), "r", encoding="utf-8") as f:
  config = json.load(f)
  config["quantization_config"]["disable_exllama"] = False
  with open(os.path.join(save_folder, "config.json"), "w", encoding="utf-8") as f:
    json.dump(config, f, indent=2)

## 4. 성능 및 추론 속도 테스트

최신 트랜스포머 릴리스에서는 `AutoModelForCausalLM` 클래스를 활용하여 GPTQ 양자화된 모델을 직접 로드할 수 있습니다. 허깅 페이스에서 이미 양자화된 모델(예: [TheBloke/Llama-2-13B-chat-GPTQ](https://huggingface.co/TheBloke/Llama-2-13B-chat-GPTQ))을 로드하거나, 직접 양자화한 모델을 로드할 수 있습니다. 여기에서는 양자화 결과를 테스트하기 위해 양자화된 모델을 디스크에서 로드하고, 비양자화 모델과 비교할 것입니다.

먼저, 비양자화된 모델을 로드하고 간단한 프롬프트로 테스트해 보겠습니다.

In [ ]:
import time 

# 프롬프트는 모델의 미세조정을 기반으로 합니다.: https://www.philschmid.de/instruction-tune-llama-2#4-test-model-and-run-inference
# prompt = """### Instruction:
# Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

# ### Input:
# Dear [boss name],

# I'm writing to request next week, August 1st through August 4th,
# off as paid time off.

# I have some personal matters to attend to that week that require
# me to be out of the office. I wanted to give you as much advance
# notice as possible so you can plan accordingly while I am away.

# Thank you, [Your name]

# ### Response:
# """

prompt = """### Instruction:
What are some common ways to deploy a model on AWS?

### Response:
"""


# 텍스트를 생성하고 지연 시간을 측정하는 헬퍼 함수
def generate_helper(pipeline,prompt=prompt):
    # 워밍업
    for i in range(5):
      _ = pipeline("Warm up")

    # 간단한 방법으로 지연 시간을 측정합니다.
    start = time.time()
    out = pipeline(prompt, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.9)
    end = time.time()
    
    generated_text = out[0]["generated_text"][len(prompt):]
    
    latency_per_token_in_ms = ((end-start)/len(pipeline.tokenizer(generated_text)["input_ids"]))*1000
    
    # 생성된 텍스트와 지연 시간을 반환합니다.
    return {"text": out[0]["generated_text"][len(prompt):], "latency": f"{round(latency_per_token_in_ms,2)}ms/token"}


우리는 기본 트랜스포머 모델을 로드하고 `pipeline` 클래스를 활용하여 추론을 실행할 수 있습니다.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# 허깅 페이스 모델 id
model_checkpoint = "NousResearch/Llama-2-7b-hf" # non-gated
#model_checkpoint = "meta/llama-2-7b" # gated

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, device_map="auto", torch_dtype=torch.float16) # we load the model in fp16 on purpose

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

기준값 생성

In [ ]:
import torch 

vanilla_res = generate_helper(pipe)

print(f"Latency: {vanilla_res['latency']}")
print(f"GPU memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Generated Instruction: {vanilla_res['text']}")

# Latency: 37.49ms/token
# GPU memory: 12.62 GB
# Generated Instruction: Write a request for PTO letter to my boss

In [ ]:
# 정리
del pipe
del model 
del tokenizer
torch.cuda.empty_cache()

이제 우리는 기준값과 동일한 프롬프트로 양자화된 모델을 테스트할 수 있습니다. 기준값이 마련되었으므로 GPTQ 양자화된 가중치를 테스트하고 검증할 수 있습니다. 이를 위해 새로운 `AutoModelForCausalLM` 클래스의 `gptq` 통합 기능을 활용하여 `gptq` 가중치를 직접 로드할 수 있습니다.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# GPTQ 가중치 경로
quantized_model_checkpoint = "quantized_llama"

q_tokenizer = AutoTokenizer.from_pretrained(quantized_model_checkpoint)
q_model = AutoModelForCausalLM.from_pretrained(quantized_model_checkpoint, device_map="auto", torch_dtype=torch.float16)

qtq_pipe = pipeline("text-generation", model=q_model, tokenizer=q_tokenizer)

이제 우리는 기준값과 동일한 프롬프트로 양자화된 모델을 테스트할 수 있습니다.

In [ ]:
gpq_res = generate_helper(qtq_pipe)

print(f"Latency: {gpq_res['latency']}")
print(f"GPU memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Generated Instruction: {gpq_res['text']}")

# Latency: 36.0ms/token
# GPU memory: 3.83 GB
# Generated Instruction: Write a letter requesting time off

비교를 위해, 기본 모델은 약 12.6GB의 메모리를 필요로 했으며, GPTQ 모델은 약 3.8GB의 메모리로 동일한 성능을 제공했습니다. GPTQ를 활용하여 약 4배의 메모리를 절약할 수 있었습니다 (파이토치의 기본 커널을 확인해야 합니다).

텍스트 생성 추론을 활용하면 토큰당 약 `22.942983ms`의 지연 시간을 달성할 수 있으며, 이는 트랜스포머보다 2배 빠릅니다. 모델을 운영 환경에 배포할 계획이 있다면, 텍스트 생성 추론을 활용하는 것을 추천합니다.